In [0]:
!pip install gensim

In [0]:
import os
import re

import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM, Dense, Embedding, Bidirectional, Dropout
from keras.layers import SpatialDropout1D, Conv1D, MaxPooling1D
from gensim.models import Word2Vec
from keras.utils import np_utils
from keras.preprocessing import sequence
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from keras.regularizers import l2
import seaborn as sns
from gensim.models import KeyedVectors
from sklearn.feature_extraction.text import TfidfVectorizer


In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
w2v_path = os.path.join("/content/gdrive/My Drive/", "Colab Notebooks/ruwikiruscorpora-nobigrams_upos_skipgram_300_5_2018.vec.gz")
hh_data_path = os.path.join("/content/gdrive/My Drive/", "Colab Notebooks/hh_dataset.csv")

In [96]:
dataset_hh = pd.read_csv(hh_data_path, sep="\t")
dataset_hh = dataset_hh.loc[dataset_hh["lang_text"] != "english"]
#dataset_hh.drop(dataset_hh.index[dataset_hh["lang_text"] == "english"], inplace=True)
dataset_hh.reset_index(drop=True, inplace=True)
dataset_hh.head()

,id,lang_title,requirement,requirement_lemmas,requirement_lemmas_tags,requirement_norm,requirement_tokens,responsibility,responsibility_lemmas,responsibility_lemmas_tags,...,title_tokens,url,lang_text,profarea_names,specializations,text,text_lemmas,text_lemmas_tags,text_normalized,text_tokens
0,29103667,russian,Высшее (желательно техническое) образование. П...,высший желательный технический образование lot...,высший_ADJ желательный_ADJ технический_ADJ обр...,высш желательн техническ образован lotus хорош...,высшее желательно техническое образование lotu...,Ежедневное планирование работы завода (молочно...,ежедневный планирование работа завод молочный ...,ежедневный_ADJ планирование_NOUN работа_NOUN з...,...,специаидать_NOUN планирование_NOUN производств...,https://api.hh.ru/vacancies/29103667?host=hh.ua,russian,"['Производство', 'Закупки']","['Пищевая промышленность', 'Управление закупка...",<p><strong>Международная компания – крупнейший...,международный компания крупнейший производител...,международный_ADJ компания_NOUN крупнейший_ADJ...,международн компан крупн производител свеж мол...,международный_ADJ компания_NOUN крупнейший_ADJ...
1,29103666,russian,Умение работать в коллективе. Образование не н...,умение работать коллектив образование ниже тех...,умение_NOUN работать_VERB коллектив_NOUN образ...,умен работа коллектив образован ниж техническ ...,умение работать коллективе образование ниже те...,Техническая подготовка выполнения сварочных ра...,технический подготовка выполнение сварочный ра...,технический_ADJ подготовка_NOUN выполнение_NOU...,...,главный_ADJ сварщик_NOUN,https://api.hh.ru/vacancies/29103666?host=hh.ua,russian,['Рабочий персонал'],['Сварщик'],<strong>Обязанности:</strong> <ul> <li><em>Тех...,обязанность технический подготовка выполнение ...,обязанность_NOUN технический_ADJ подготовка_NO...,обязан техническ подготовк выполнен сварочн ра...,обязанность_NOUN технический_ADJ подготовка_NO...
2,29103664,russian,Профильное высшее образование (Художественное/...,профильный высший образование креативность нес...,профильный_ADJ высший_ADJ образование_NOUN кре...,профильн высш образован креативн нестандартн м...,профильное высшее образование креативность нес...,Дизайн и верстка еженедельного акционного букл...,дизайн верстка еженедельный акциоть буклет раз...,дизайн_NOUN верстка_NOUN еженедельный_ADJ акци...,...,дизайнер_NOUN,https://api.hh.ru/vacancies/29103664?host=hh.ua,russian,"['Искусство, развлечения, масс-медиа', 'Маркет...","['Дизайн, графика, живопись', 'Дизайнер']",<p><strong>Сеть гипермаркетов ProStore приглаш...,сеть гипермаркет prostore приглашать постоянны...,сеть_NOUN гипермаркет_NOUN prostore_X приглаша...,сет гипермаркет prostor приглаша постоя работ ...,сеть_NOUN гипермаркет_NOUN prostore_X приглаша...
3,29103661,russian,1. Высшее образование ( в приоритете: менеджме...,высший образование приоритет менеджмент маркет...,высший_ADJ образование_NOUN приоритет_VERB мен...,высш образован приоритет менеджмент маркетинг ...,высшее образование приоритете менеджмент марке...,Наша студия помогает компаниям повышать продаж...,наш студия помогать компания повышать продажа ...,наш_DET студия_NOUN помогать_VERB компания_NOU...,...,руководитель_NOUN отдел_NOUN продажа_NOUN,https://api.hh.ru/vacancies/29103661?host=hh.ua,russian,"['Продажи', 'Продажи', 'Высший менеджмент']","['Услуги для бизнеса', 'Управление продажами',...",<p>Наша студия помогает компаниям повышать про...,наш студия помогать компания повышать продажа ...,наш_DET студия_NOUN помогать_VERB компания_NOU...,наш студ помога компан повыша продаж помощ про...,наш_DET студия_NOUN помогать_VERB компания_NOU...
4,29103660,russian,Требования: Опыт работы по специальности обяза...,требование опыт работа специальность обязатель...,требование_NOUN опыт_NOUN работа_NOUN специаль...,требован оп работ специальн обязател желательн...,требования опыт работы специальности обязателе...,Обязанности: Заточка и доводка режущего инстру...,обязанность заточка доводка рез

In [0]:
vectorizer = KeyedVectors.load_word2vec_format(w2v_path, binary=False)

### Prepare dataset

In [0]:
def get_mask(text, req):
    
    text = text.split(" ")
    req = req.split(" ")
    mask = np.zeros(len(text))
    
    i = 0
    while i  <= (len(text)):      
                   
        if (req == text[i: i + len(req)]):
            for j in range(len(req)):
                mask[i + j] = 1
            i += 1
            break

        i += 1
    return mask 

In [0]:
def get_training_sample(model, text):
    """
    Gets 1 sample of training data
    
    :param  model text:
    :return str text:
    """
    tagged_list = text.split(" ")

    vec_list = []
    
    #converting word2vec 
    for word in tagged_list:
        try:
            vec_list.append(model[word])
        except:
            #print("Word " + word + " isn't in vocab. Embeding as zeros")
            vec_list.append(np.zeros(300))
    return vec_list
  

In [65]:
text = dataset_hh.loc[0, "text_lemmas_tags"]
print(text)
sub_text = dataset_hh.loc[0, "requirement_lemmas_tags"]
print(sub_text)

международный_ADJ компания_NOUN крупнейший_ADJ производитель_NOUN свежий_ADJ молочный_ADJ продукт_NOUN бутилировать_VERB минеральный_ADJ вода_NOUN детский_ADJ клинический_ADJ питание_NOUN приглашать_VERB работа_NOUN специалист_NOUN планирование_NOUN производство_NOUN обязанность_NOUN ежедневный_ADJ планирование_NOUN работа_NOUN завод_NOUN молочный_ADJ производство_NOUN основа_NOUN спланировать_ADJ производство_NOUN формирование_NOUN график_NOUN работа_NOUN оборудование_NOUN оценка_NOUN потребность_NOUN молочный_ADJ сырье_NOUN ежедневный_ADJ недельный_ADJ месячный_ADJ планирование_NOUN объем_NOUN закупка_NOUN дополнительный_ADJ сырье_NOUN либо_CCONJ реализация_NOUN избыточный_ADJ случай_NOUN необходимость_NOUN планирование_NOUN поставка_NOUN упаковочный_ADJ материал_NOUN ингредиетовый_NOUN потребность_NOUN производство_NOUN требование_NOUN высший_ADJ желательный_ADV технический_ADJ образование_NOUN lotus_X хороший_ADJ знание_NOUN excel_X английский_ADJ elementary_X опционально_ADV ответ

In [0]:
x = get_training_sample(vectorizer, text)

In [0]:
text = dataset_hh.loc[0, "text_normalized"]
sub_text = dataset_hh.loc[0, "requirement_norm"]

y = get_mask(text, sub_text)

In [115]:
print(len(x))
print(len(y))
print(y)
y = sequence.pad_sequences([list(y)], maxlen=50, dtype='float', padding="pre", truncating="pre")
print()
print(y)

83
83
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0.
  0. 0.]]


In [168]:
X = []
Y = []
max_len_real = 0
max_len = 300
for i, row in dataset_hh.iterrows():
  if not isinstance(row["requirement_norm"], str):
    continue
  text = row["text_lemmas_tags"]
  x_sample = get_training_sample(vectorizer, text)

  text = row["text_lemmas"]
  sub_text = row["requirement_lemmas"]
  y_sample = get_mask(text, sub_text)
  
  Y.append(y_sample)
  X.append(x_sample)
  
  if (len(x_sample) != len(y_sample)):
    print(len(x_sample))
    print(len(y_sample))
    print(len(row["text_normalized"].split(" ")))
    print(y_sample)
    print(i)
  assert(len(x_sample) == len(y_sample))
  
  if len(y_sample) > max_len_real:
    max_len_real = len(y_sample)
    
X = sequence.pad_sequences(X, maxlen=max_len, dtype='float', padding="post", truncating="pre")
Y = sequence.pad_sequences(Y, maxlen=max_len, dtype='float', padding="post", truncating="pre")
       
Y = np.array(Y)
Y = np.reshape(Y, (Y.shape[0], Y.shape[1], -1))
X = np.array(X)
print("Max len =", max_len)
print("Max len edited =", padding)

Max len = 300
Max len edited = 300


In [169]:
print(X.shape)
print(Y.shape)

(3472, 300, 300)
(3472, 300, 1)


### seq2seq model

In [0]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense
from keras.utils import to_categorical

In [171]:
Y_new = np.zeros((Y.shape[0], Y.shape[1], 2))
for i in range(Y.shape[0]):
  for j in range(Y.shape[1]):
    Y_new[i, j] = to_categorical(Y[i, j, 0], num_classes=2)
    
Y = Y_new
print(Y.shape)

(3472, 300, 2)


In [0]:
encoder_input_data = X
decoder_input_data = Y

decoder_target_data = np.zeros(Y.shape)

for i in range(Y.shape[0]):
  for j in range(0, Y.shape[1]-1):
    decoder_target_data[i, j] =  Y[i, j + 1]

In [176]:

for i in range(55, 75):
    print('Y:', Y[0, i], 'Target', decoder_target_data[0, i])

Y: [1. 0.] Target [1. 0.]
Y: [1. 0.] Target [0. 1.]
Y: [0. 1.] Target [0. 1.]
Y: [0. 1.] Target [0. 1.]
Y: [0. 1.] Target [0. 1.]
Y: [0. 1.] Target [0. 1.]
Y: [0. 1.] Target [0. 1.]
Y: [0. 1.] Target [0. 1.]
Y: [0. 1.] Target [0. 1.]
Y: [0. 1.] Target [0. 1.]
Y: [0. 1.] Target [0. 1.]
Y: [0. 1.] Target [0. 1.]
Y: [0. 1.] Target [0. 1.]
Y: [0. 1.] Target [0. 1.]
Y: [0. 1.] Target [0. 1.]
Y: [0. 1.] Target [0. 1.]
Y: [0. 1.] Target [0. 1.]
Y: [0. 1.] Target [0. 1.]
Y: [0. 1.] Target [1. 0.]
Y: [1. 0.] Target [1. 0.]


In [0]:
num_encoder_tokens = 300
latent_dim = 200

num_decoder_tokens = 2
max_decoder_seq_length = max_len

batch_size = 64
epochs = 100

In [0]:

# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the 
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [0]:
# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', loss_weights=[[0.1, 1000_000]], metrics=['accuracy'])


In [285]:
history = model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2,
          verbose=1)
#  class_weight={0: 1.0, 1: 1000}

Train on 2777 samples, validate on 695 samples
Epoch 1/100
2777/2777 [==============================] - 121s 44ms/step - loss: 56429.0898 - acc: 0.9728 - val_loss: 29329.8496 - val_acc: 0.9809
Epoch 2/100
2777/2777 [==============================] - 115s 41ms/step - loss: 20854.0352 - acc: 0.9877 - val_loss: 19926.9395 - val_acc: 0.9872
Epoch 3/100
2777/2777 [==============================] - 117s 42ms/step - loss: 15723.0869 - acc: 0.9913 - val_loss: 14614.7988 - val_acc: 0.9907
Epoch 4/100
2777/2777 [==============================] - 116s 42ms/step - loss: 12806.6875 - acc: 0.9935 - val_loss: 12397.9463 - val_acc: 0.9940
Epoch 5/100
2777/2777 [==============================] - 116s 42ms/step - loss: 11558.0371 - acc: 0.9948 - val_loss: 12796.9033 - val_acc: 0.9940
Epoch 6/100
2777/2777 [==============================] - 117s 42ms/step - loss: 11735.7852 - acc: 0.9947 - val_loss: 12642.3145 - val_acc: 0.9940
Epoch 7/100
2777/2777 [==============================] - 116s 42ms/step - los

In [0]:
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

In [0]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, 0] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = []
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        """sampled_char = reverse_target_char_index[sampled_token_index]"""
        decoded_sentence.append(sampled_token_index)

        # Exit condition: either hit max length
        # or find stop character.
        if (len(decoded_sentence) == max_decoder_seq_length) or \
        (len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

### Test

In [312]:
dataset_hh.loc[146, "requirement_lemmas"]

'уверенный знание java eclipse intellij опыт работа лета область знание реляционный субд свободный владение'

In [0]:
predict = decode_sequence(X[146].reshape((1, max_len, 300)))

In [310]:
for i in range(0, 300):
    print('Expected:', np.argmax(Y[146, i]), 'Predicted', predict[i])

Expected: 0 Predicted 0
Expected: 0 Predicted 0
Expected: 0 Predicted 0
Expected: 0 Predicted 0
Expected: 0 Predicted 0
Expected: 0 Predicted 0
Expected: 0 Predicted 0
Expected: 0 Predicted 0
Expected: 0 Predicted 0
Expected: 0 Predicted 0
Expected: 0 Predicted 0
Expected: 0 Predicted 0
Expected: 0 Predicted 0
Expected: 0 Predicted 0
Expected: 0 Predicted 0
Expected: 0 Predicted 0
Expected: 0 Predicted 0
Expected: 0 Predicted 0
Expected: 0 Predicted 0
Expected: 0 Predicted 0
Expected: 0 Predicted 0
Expected: 0 Predicted 0
Expected: 0 Predicted 0
Expected: 0 Predicted 0
Expected: 0 Predicted 0
Expected: 0 Predicted 0
Expected: 0 Predicted 0
Expected: 0 Predicted 0
Expected: 0 Predicted 0
Expected: 0 Predicted 0
Expected: 0 Predicted 0
Expected: 0 Predicted 0
Expected: 1 Predicted 0
Expected: 1 Predicted 0
Expected: 1 Predicted 0
Expected: 1 Predicted 0
Expected: 1 Predicted 0
Expected: 1 Predicted 0
Expected: 0 Predicted 0
Expected: 0 Predicted 0
Expected: 0 Predicted 0
Expected: 0 Pred

In [290]:
%%time
#X.shape[0]
Y_predict = []
for i in range(X.shape[0]):
  predict = decode_sequence(X[i].reshape((1, max_len, 300)))
  Y_predict.append(predict)
  

CPU times: user 25min 7s, sys: 1min 2s, total: 26min 10s
Wall time: 16min 56s


In [291]:
Y_predict = np.array(Y_predict)
Y_predict.shape

(3472, 300)

In [301]:
np.sum(Y[:, :, 1])

34050.0

In [299]:
np.sum(Y_predict)

3199

In [295]:
count = 0 
for i in range(Y_predict.shape[0]):
  if np.sum(Y_predict[i, :]) != 0:
    print(" I =", i, "____________")
    print(Y_predict[i])
    count += 1
print("total =", count)

 I = 50 ____________
[0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0]
 I = 145 ____________
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 